# Purpose

This notebook just contains a bunch of cells that I used to explore the model and some alternatives. 
It is mostly not directly useful to producing the experiments.

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import argparse

import numpy as np
import os
import pandas as pd
import scipy as sp
import sys
import torch
import torch.nn.functional as F
import warnings
import random
import collections

# CD-T Imports
import math
import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import itertools

from torch import nn

warnings.filterwarnings("ignore")

base_dir = os.path.split(os.getcwd())[0]
sys.path.append(base_dir)

from argparse import Namespace
from methods.bag_of_ngrams.processing import cleanReports, cleanSplit, stripChars
from pyfunctions.general import extractListFromDic, readJson, combine_token_attn, compute_word_intervals
from pyfunctions.pathology import extract_synoptic, fixLabelProstateGleason, fixProstateLabels, fixLabel, exclude_labels
from pyfunctions.cdt_basic import *
from pyfunctions.ioi_dataset import IOIDataset
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import AutoTokenizer, AutoModel
from transformers import GPT2Tokenizer, GPT2Model

In [4]:
torch.autograd.set_grad_enabled(False)

## Load Model

Note: Unlike with the BERT model + medical dataset objective, it is not necessary to pretrain GPT-2 to perform the IOI dataset.
GPT-2-small is already capable of performing IOI; that's part of the point of the Mech Interp in the Wild paper.
We only need to examine how it does it.

In [7]:
device = 'cpu'

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')

In [7]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


OrderedDict([('transformer',
              GPT2Model(
                (wte): Embedding(50257, 768)
                (wpe): Embedding(1024, 768)
                (drop): Dropout(p=0.1, inplace=False)
                (h): ModuleList(
                  (0-11): 12 x GPT2Block(
                    (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
                    (attn): GPT2SdpaAttention(
                      (c_attn): Conv1D()
                      (c_proj): Conv1D()
                      (attn_dropout): Dropout(p=0.1, inplace=False)
                      (resid_dropout): Dropout(p=0.1, inplace=False)
                    )
                    (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
                    (mlp): GPT2MLP(
                      (c_fc): Conv1D()
                      (c_proj): Conv1D()
                      (act): NewGELUActivation()
                      (dropout): Dropout(p=0.1, inplace=False)
                    )
                  )
      

In [35]:
text = "Replace me by any text you'd"
input = tokenizer(text, return_tensors='pt').input_ids
# print(encoded_input) # has 'input_idx' and 'attention_mask'
# output = model(input)
# print(output.last_hidden_state.shape)
gen_tokens = model.generate(input, pad_token_id=tokenizer.pad_token_id, output_scores=True)
print(gen_tokens)
gen_text = tokenizer.batch_decode(gen_tokens)
gen_text

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[3041, 5372,  502,  416,  597, 2420,  345, 1549,  588,   13,  198,  198,
           40, 1101,  407, 1654,  611,  345,  821, 3910]])


/Users/georgiazhou/miniconda3/lib/python3.12/site-packages/transformers/generation/utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


["Replace me by any text you'd like.\n\nI'm not sure if you're aware"]

In [21]:
# other exploratory stuff
# model.to_tokens(text) #turns out this is a utility of trasnformer_lens
# print(output.past_key_values[0][0].shape) # this has to do with key matrix stuff
#print(output.values())
#output.logits

AttributeError: 'BaseModelOutputWithPastAndCrossAttentions' object has no attribute 'logits'

In [21]:
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 320.7 kB/s eta 0:00:00:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.4/392.4 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.2/27.2 MB 5.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.2
    Uninstalling tqdm-4.66.

In [22]:
from datasets import load_dataset
ioi_dataset = load_dataset("fahamu/ioi")
# i've decided against using this for the most part; it's better to use the raw IOIDataset 
# from the paper and from the related notebook using EasyTransformers, since these both provide many utilities for dealing with the data

Generating train split: 100%|██████████| 26210000/26210000 [00:03<00:00, 7827513.55 examples/s]


In [12]:
!pip install transformer_lens
!pip install einops

In [5]:
# Model code adapted from Callum McDougall's notebook for ARENA on reproducing the IOI paper using TransformerLens.
# This makes some sense, since EasyTransformer, the repo/lib released by the IOI guys, was forked from TransformerLens.
# In fact, this makes the reproduction a little bit more faithful, since they most likely do certain things such as 
# "folding" LayerNorms to improve their interpretability results, and we are able to do the same by using TransformerLens.
# HuggingFace, by contrast, has the most impenetrable docs and tons of outdated APIs and etc.; even their source 
# code is impossible to traverse, and I gave up on it, thankfully quickly.

from transformer_lens import utils, HookedTransformer, ActivationCache
model = HookedTransformer.from_pretrained("gpt2-small",
                                          center_unembed=True,
                                          center_writing_weights=True,
                                          fold_ln=False,
                                          refactor_factored_attn_matrices=True)
                                          

Loaded pretrained model gpt2-small into HookedTransformer


## Example forward pass

In [24]:
text = "After John and Mary went to the store, John gave a bottle of milk to"
tokens = model.to_tokens(text).to(device)
logits, cache = model.run_with_cache(tokens)
probs = logits.softmax(dim=-1)
most_likely_next_tokens = model.tokenizer.batch_decode(logits.argmax(dim=-1)[0])

In [26]:
for activation_name, activation in cache.items():
    # Only print for first layer
    if ".0." in activation_name or "blocks" not in activation_name:
        print(f"{activation_name:30} {tuple(activation.shape)}")

hook_embed                     (1, 17, 768)
hook_pos_embed                 (1, 17, 768)
blocks.0.hook_resid_pre        (1, 17, 768)
blocks.0.ln1.hook_scale        (1, 17, 1)
blocks.0.ln1.hook_normalized   (1, 17, 768)
blocks.0.attn.hook_q           (1, 17, 12, 64)
blocks.0.attn.hook_k           (1, 17, 12, 64)
blocks.0.attn.hook_v           (1, 17, 12, 64)
blocks.0.attn.hook_attn_scores (1, 12, 17, 17)
blocks.0.attn.hook_pattern     (1, 12, 17, 17)
blocks.0.attn.hook_z           (1, 17, 12, 64)
blocks.0.hook_attn_out         (1, 17, 768)
blocks.0.hook_resid_mid        (1, 17, 768)
blocks.0.ln2.hook_scale        (1, 17, 1)
blocks.0.ln2.hook_normalized   (1, 17, 768)
blocks.0.mlp.hook_pre          (1, 17, 3072)
blocks.0.mlp.hook_post         (1, 17, 3072)
blocks.0.hook_mlp_out          (1, 17, 768)
blocks.0.hook_resid_post       (1, 17, 768)
ln_final.hook_scale            (1, 17, 1)
ln_final.hook_normalized       (1, 17, 768)


In [8]:
# hack to get model dtype out, for compatibility with other code
next(model.parameters()).dtype

torch.float32

In [16]:
print(model)
# print(model.config) # doesn't work on hookedtransformer, is a huggingface thing
# print(model.embed.dtype) same, but can use dtype trick
# print(type(model))
#model.state_dict().keys()#.blocks[0].mlp

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (ln1): LayerNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_re

In [19]:
import inspect
# inspect.getclasstree(inspect.getmro(type(model)))
inspect.getmro(type(model))

(transformer_lens.HookedTransformer.HookedTransformer,
 transformer_lens.hook_points.HookedRootModule,
 torch.nn.modules.module.Module,
 object)

In [18]:
import torchviz
dir(model)
# torchviz.make_dot(model)
# model._modules

['OV',
 'QK',
 'T_destination',
 'W_E',
 'W_E_pos',
 'W_K',
 'W_O',
 'W_Q',
 'W_U',
 'W_V',
 'W_gate',
 'W_in',
 'W_out',
 'W_pos',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_compiled_call_impl',
 '_enable_hook',
 '_enable_hook_with_name',
 '_enable_hooks_for_points',
 '_forward_hooks',
 '_forward_hooks_always_called',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_init_weights_gpt2',
 '_in

In [23]:
!pip install torchsummary
!pip install torchinfo

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
import pdb
from torchinfo import summary

text = "After John and Mary went to the store, John gave a bottle of milk to"
encoding = get_encoding(text, model.tokenizer, "cpu")
# embedding_output = model.embed(encoding.input_ids)
input_shape = encoding.input_ids.shape
print(input_shape)
pdb.set_trace()
summary(model, input_shape, device='cpu')


torch.Size([1, 512])
--Return--
None
> /var/folders/c9/mvmx0pt17m51nyh8wgjnqj900000gn/T/ipykernel_11966/1150960534.py(9)<module>()
      6 # embedding_output = model.embed(encoding.input_ids)
      7 input_shape = encoding.input_ids.shape
      8 print(input_shape)
----> 9 pdb.set_trace()
     10 summary(model, input_shape, device='cpu')

Moving model to device:  cpu
> /Users/georgiazhou/miniconda3/lib/python3.12/site-packages/transformer_lens/HookedTransformer.py(332)input_to_embed()
    330             tokens = self.hook_tokens(tokens)
    331         pdb.set_trace()
--> 332         embed = self.hook_embed(self.embed(tokens))  # [batch, pos, d_model]
    333         if self.cfg.positional_embedding_type == "standard":
    334             pos_embed = self.hook_pos_embed(

tensor([[0.4899, 0.2857, 0.5669, 0.1953, 0.6854, 0.5814, 0.6580, 0.0523, 0.2990,
         0.1988, 0.7593, 0.1852, 0.7425, 0.4648, 0.5173, 0.7029, 0.3675, 0.9153,
         0.7823, 0.3810, 0.3304, 0.9647, 0.0897, 0.223

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []

In [33]:
# Same as in the notebook, example
example_prompt = "After John and Mary went to the store, John gave a bottle of milk to"
example_answer = "Mary"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)o

Tokenized prompt: ['<|endoftext|>', 'After', ' John', ' and', ' Mary', ' went', ' to', ' the', ' store', ',', ' John', ' gave', ' a', ' bottle', ' of', ' milk', ' to']
Tokenized answer: [' Mary']


Performance on answer token:
Rank: 0        Logit: 18.09 Prob: 70.07% Token: | Mary|

Top 0th token. Logit: 18.09 Prob: 70.07% Token: | Mary|
Top 1th token. Logit: 15.38 Prob:  4.67% Token: | the|
Top 2th token. Logit: 15.35 Prob:  4.54% Token: | John|
Top 3th token. Logit: 15.25 Prob:  4.11% Token: | them|
Top 4th token. Logit: 14.84 Prob:  2.73% Token: | his|
Top 5th token. Logit: 14.06 Prob:  1.24% Token: | her|
Top 6th token. Logit: 13.54 Prob:  0.74% Token: | a|
Top 7th token. Logit: 13.52 Prob:  0.73% Token: | their|
Top 8th token. Logit: 13.13 Prob:  0.49% Token: | Jesus|
Top 9th token. Logit: 12.97 Prob:  0.42% Token: | him|


Ranks of the answer tokens: [(' Mary', 0)]

## Generate dataset/Explore types

In [16]:
data = IOIDataset(N=500, prompt_type="ABBA", tokenizer=model.tokenizer)
#data.tokenized_prompts
data.ioi_prompts[0]
[x['TEMPLATE_IDX'] for x in data.ioi_prompts[0:10]]

[12, 4, 9, 6, 1, 13, 2, 8, 8, 4]

In [16]:
# test
pos_specific_hs = [
        [i for i in range(12)],
        [0],
        [i for i in range(12)]
    ]
all_heads = list(itertools.product(*pos_specific_hs))
target_nodes = [(7, 82, 11), (7, 82, 0), (7, 82, 6), (9, 82, 0), (9, 91, 7), (8, 82, 0)] # not meaningful in a GPT context
source_list = [[node] for node in all_heads if node not in target_nodes]

text = "After John and Mary went to the store, John gave a bottle of milk to"
encoding = get_encoding(text, model.tokenizer, device)
# encoding.input_ids.shape # 512-long vector, not sure why the tokens change from EOS to 0 at some point
# embedding = model.embed(encoding.input_ids)

out_decomps, target_decomps = prop_model_hh_batched(encoding, model, source_list, target_nodes,
                                                                   device=device,
                                                                   patched_values=None, mean_ablated=False, num_at_time=1)
                                                                   # patched_values=mean_act, mean_ablated=True)
                                                                

AssertionError: # indices in 'batch query_pos d_model' != length of shape [1, 512, 768]

## Explore IOI Dataset


In [11]:
from pyfunctions.ioi_dataset import IOIDataset

ioi_dataset = IOIDataset(prompt_type="mixed", N=50, tokenizer=model.tokenizer, prepend_bos=False)

In [40]:
ioi_dataset.toks.shape

ioi_dataset.word_idx

ioi_dataset.sentences[:4]

ioi_dataset.groups

# ioi_dataset.toks[ioi_dataset.groups[-1]]
# [ioi_dataset.sentences[x] for x in ioi_dataset.groups[3]] # sentences of the same group are identical except for the choice of nouns

[array([ 4, 16, 23, 42, 48]),
 array([33, 34]),
 array([36]),
 array([ 0,  9, 19, 25, 29, 38, 39]),
 array([ 1,  5, 35, 49]),
 array([ 3, 47]),
 array([ 2,  7, 26, 40, 44]),
 array([18]),
 array([32, 43, 45]),
 array([20, 24, 30]),
 array([ 6, 12, 37]),
 array([ 8, 22]),
 array([11, 13, 14, 15, 17, 21, 27, 31]),
 array([10, 28, 41, 46])]

In [38]:
# The below is wrong! The generated sentences are not of the same format as what is described in the paper, and 
# this is also not what they do in their experiments.py.
# abc_dataset = IOIDataset(prompt_type="ABC mixed", N=50, tokenizer=model.tokenizer, prepend_bos=False)

# Instead, do this, apparently.
abc_dataset = (
    ioi_dataset.gen_flipped_prompts(("IO", "RAND"))
    .gen_flipped_prompts(("S", "RAND"))
    .gen_flipped_prompts(("S1", "RAND"))
) # Note generating several of these in a row will generate different random names; this can be useful for a quick mean ablation.

In [39]:
abc_dataset.sentences[:4]

['Then, Katie and Angela were working at the hospital. Nicole decided to give a computer to Angela',
 'Then, Jessica and Sarah went to the hospital. Jose gave a kiss to Sarah',
 'When Jeremy and Jonathan got a basketball at the garden, Jesse decided to give it to Jonathan',
 'Then, Tyler and Kristen were thinking about going to the hospital. Christopher wanted to give a bone to Tyler']